In [1]:
from Dataload import dataload

from module import transformer,video_model,invres

import torch
import torch.nn as nn
from torchsummary import summary
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt
from torchvision import  utils
import os
from constant import EMOTIPATH
import time
from tqdm.notebook import tqdm


In [2]:
Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_video=os.path.join(EMOTIPATH,"Train")
Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Val_video=os.path.join(EMOTIPATH,"Val")

frame_num=1
train_data=dataload.Video_Frame_Data(Train_label,base_path_v=Train_video,frame_num=frame_num)



In [3]:
train_data[0][1]

['000011.png', '000008.png', '000004.png', '000003.png', '000016.png', '000025.png', '000005.png', '000002.png', '000017.png', '000026.png', '000014.png', '000010.png', '000001.png', '000021.png', '000013.png', '000007.png', '000019.png', '000009.png', '000022.png', '000006.png', '000020.png', '000015.png', '000012.png', '000024.png', '000018.png', '000023.png']
['000001.png', '000002.png', '000003.png', '000004.png', '000005.png', '000006.png', '000007.png', '000008.png', '000009.png', '000010.png', '000011.png', '000012.png', '000013.png', '000014.png', '000015.png', '000016.png', '000017.png', '000018.png', '000019.png', '000020.png', '000021.png', '000022.png', '000023.png', '000024.png', '000025.png', '000026.png']


array([[0.]])

In [ ]:
#Batchfy
#Counts number of correct label
def num_correct(prediction,labels):
    correct=0
    for i,(pred_label,label) in enumerate(zip(prediction,labels)):
        if (pred_label.item()==label.item()):
            correct +=1
    return correct


valid_data=dataload.Video_Frame_Data(Val_labels,"valid.csv",base_path_v=Val_video,frame_num=frame_num)

#train_data1=dataload.Video_Embedding_Data("train.csv",Train_label)
#valid_data=dataload.Video_Embedding_Data("valid.csv",Val_labels)
            
train_dataloader = DataLoader(train_data, batch_size=32
                       , num_workers=0,shuffle=True)
#valid_data=dataload.Video_Frame_Data(Val_labels,base_path_v=Val_video,strict_num=26)

valid_dataloader = DataLoader(valid_data, batch_size=32
                       , num_workers=0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)
model=video_model.Encoder(num_frame=frame_num,isframe=1)
for name, child in model.named_children():
   if not name in ['densenet']:
       print(name + ' is unfrozen')
       for param in child.parameters():
           param.requires_grad = True
   else:
       print(name + ' is frozen')
       for param in child.parameters():
           param.requires_grad = False
#model=model.to(device)  
model=model.double().to(device)
#model=model.float()
criterion = nn.CrossEntropyLoss()  
loss_func = torch.nn.MSELoss()

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0002, betas=(0.5, 0.999))

len(train_data)

In [ ]:
def train(num_epochs,name,model):
    f = open(name+".txt",'a')
    start = time.time()
    #Triaining
    train_loss=[]
    valid_accuracy=[]
    model.train()
    for epochs in range(0,num_epochs):
        first=True
        model.train()
        correct=0
        total_samples=0
        avg_tloss=0
        print("Training Epoch: ", epochs+1,"\n")
        for i_batch, (sample_batched,_,label) in tqdm(enumerate(train_dataloader)):
           # sample_batched=sample_batched.frames
            batch_size=sample_batched.size(0)
            optimizer.zero_grad()
            sample_batched=sample_batched.to(device)
            output=model(sample_batched)
            loss=criterion(output,label.to(device))
            loss.backward()
            predicted = torch.max(output, 1)
            prediction=predicted.indices.detach().cpu()
            correct +=num_correct(prediction,label)
            total_samples+=batch_size
            accuracy=correct/(total_samples)
            optimizer.step()
            true_label=label.detach().cpu()
            avg_tloss+=loss.item()
            if first:
                first=False
                conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
            else:
                conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
            if i_batch%20==0:
               # print(label)
                print("Batch: ",i_batch+1,"/",len(train_dataloader))
                print("Batch Recognition loss: ", loss.item())

        print(conf_mat)
        avg_tloss=avg_tloss/len(train_dataloader)
        avg_taccuracy=correct/total_samples
        print("Average_Loss: ",avg_tloss)
        print("Average_Accuracy: ",avg_taccuracy)

        torch.save(model.state_dict(),name+".pth")
        print("Validation\n")
        model.eval()   
        correct=0
        total_samples=0
        avg_vloss=0
        first=True

        for i_batch, (sample_batched,_,label) in enumerate(valid_dataloader):
            batch_size=sample_batched.size(0)
            sample_batched=sample_batched.to(device)
            output=model(sample_batched)
            loss=criterion(output,label.to(device))
            avg_vloss+=loss.item()
            predicted = torch.max(output, 1)
            prediction=predicted.indices.detach().cpu()
            correct +=num_correct(prediction,label)
            total_samples+=batch_size

            true_label=label.detach().cpu()
            if first:
                first=False
                conf_mat=confusion_matrix( true_label,prediction,labels=[0,1,2])
            else:
                conf_mat+=confusion_matrix(true_label,prediction,labels=[0,1,2])
        print(conf_mat)
        avg_vloss=avg_vloss/len(valid_dataloader)
        print(avg_vloss)
        avg_vaccuracy=correct/(total_samples)
        print("Accuracy: ", avg_vaccuracy)

        data = " %f,%f,%f,%f \n" % (avg_tloss,avg_taccuracy,avg_vloss,avg_vaccuracy)
        f.write(data)



    f.close()


In [ ]:
model_name="pre_embedded_1_frames"
train(20,model_name,model)

In [ ]:
print("finished")

In [ ]:
valid_data1=dataload.Video_Frame_Data(Val_labels,base_path_v=Val_video,strict_num=26)

valid_dataloader1 = DataLoader(valid_data1, batch_size=32
                       , num_workers=0)

In [ ]:
debug

In [ ]:
train(1)